In [3]:
import numpy as np
from initLayers import initLayers
from train import train
from rna import rna
from utils import normalize, splitData, evaluate_multiclass_with_one_hot

Carrega dados do dataset

In [4]:
DATASET_PATH = 'penguins.csv'
data = np.genfromtxt(DATASET_PATH, delimiter=',', skip_header=1)

Separa features e labels

In [5]:
features = data[:, :-3]
labels = data[:, -3:]

Separa dados para treinamento e para validação

In [6]:
featuresTrain, featuresVal, labelsTrain, labelsVal = splitData(features, labels)

Normaliza features

In [7]:
normalized_features, feature_scaler = normalize(featuresTrain)

Combina features e labels e formata observações para treinamento

In [10]:
normalized_data = np.hstack((normalized_features, labelsTrain))
observations = [(row[:-labelsTrain.shape[1]], row[-labelsTrain.shape[1]:]) for row in normalized_data]

Define a estrutura da rede, funções de ativação, funções de custo, épocas e learning rate

In [11]:
# Obtém a quantidade de neurons da camada de input
INPUT_NEURONS = len(observations[0][0])

# Define a estrutura da rede e funções de ativação
layers = [
    (INPUT_NEURONS, 'RELU'),
    #(INPUT_NEURONS, 'RELU'),
    (3, 'SOFTMAX')
]

# Definição função de custo, épocas e learning rate
COSTF = "CATEGORICAL_ENTROPY"
EPOCHS = 50
LEARNING_RATE = 0.001

Inicializa os pesos e as funções das camadas

In [12]:
layers = initLayers(layers, INPUT_NEURONS)

Dispara o treinamento da rede

In [17]:
trainedParams = train(EPOCHS, LEARNING_RATE, layers, observations, COSTF)

Normaliza as features para validação

In [18]:
test_normalized_features = feature_scaler.transform(featuresVal)


Testa a rede treinada

In [19]:
predictions = []
for i, observation in enumerate(test_normalized_features):

  input = observation
  prediction = rna(input, trainedParams)

  one_hot_prediction = np.zeros_like(prediction)
  one_hot_prediction[np.argmax(prediction)] = 1

  predictions.append(one_hot_prediction)

Obtém métricas de avaliação

In [20]:
metrics = evaluate_multiclass_with_one_hot(labelsVal, predictions)

Evaluation Metrics:
accuracy: 0.9855
precision: 0.9860
confusion_matrix:
[[29  0  0]
 [ 1 10  0]
 [ 0  0 29]]
